In [1]:
pip install numpy pandas matplotlib xgboost scikit-learn imbalanced-learn shap reportlab ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from io import BytesIO
import datetime
import re
import os
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink

# Ensure plots display inline in Jupyter
%matplotlib inline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [3]:
# ─────────────────────────────────────────
# Load / Train Model Logic
# ─────────────────────────────────────────
def load_or_train_model():
    from sklearn.model_selection import train_test_split
    from xgboost import XGBClassifier
    from imblearn.over_sampling import SMOTE

    # 1. Try loading existing model
    if os.path.exists("model.pkl"):
        return joblib.load("model.pkl"), True

    # 2. If no model, check for training data
    if not os.path.exists("cs-training.csv"):
        print("ℹ️ 'cs-training.csv' not found. ML features will be disabled, but rule-based logic will work.")
        return None, False

    print("Training model... this may take a moment.")
    
    # 3. Train model
    df = pd.read_csv("cs-training.csv", index_col=0)
    df.dropna(subset=["MonthlyIncome", "NumberOfDependents"], inplace=True)
    for col in ["RevolvingUtilizationOfUnsecuredLines", "DebtRatio", "MonthlyIncome"]:
        df[col] = df[col].clip(upper=df[col].quantile(0.99))
    df = df[df["age"] >= 18]

    df["TotalLate"] = (
        df["NumberOfTime30-59DaysPastDueNotWorse"] +
        df["NumberOfTime60-89DaysPastDueNotWorse"] +
        df["NumberOfTimes90DaysLate"]
    )
    df["DelinquencyScore"] = (
        df["NumberOfTime30-59DaysPastDueNotWorse"] * 1 +
        df["NumberOfTime60-89DaysPastDueNotWorse"] * 2 +
        df["NumberOfTimes90DaysLate"]              * 3
    )
    df["DebtToIncome"]       = df["DebtRatio"] / (df["MonthlyIncome"] + 1)
    df["IncomePerDependent"] = df["MonthlyIncome"] / (df["NumberOfDependents"] + 1)

    X = df.drop(columns=["SeriousDlqin2yrs"])
    y = df["SeriousDlqin2yrs"]

    X_train, _, y_train, _ = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )
    
    # Handle imbalance
    X_res, y_res = SMOTE(random_state=42).fit_resample(X_train, y_train)

    model = XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.03,
        subsample=0.8, colsample_bytree=0.8, min_child_weight=5,
        reg_lambda=2.0, random_state=42, verbosity=0, eval_metric="logloss"
    )
    model.fit(X_res.values, y_res)
    joblib.dump(model, "model.pkl")
    print("✅ Model trained and saved as 'model.pkl'.")
    return model, True

# Initialize Model
model, model_loaded = load_or_train_model()

ℹ️ 'cs-training.csv' not found. ML features will be disabled, but rule-based logic will work.


In [4]:
# ─────────────────────────────────────────
# Risk Logic & Helpers
# ─────────────────────────────────────────
def validate_pan(pan):
    return bool(re.match(r"^[A-Z]{5}[0-9]{4}[A-Z]{1}$", pan.strip().upper()))

def compute_credit_score(prob):
    return max(300, min(850, int(850 - prob * 550)))

def score_band(score):
    if score >= 750: return "Excellent", "🟢"
    if score >= 700: return "Good",      "🟡"
    if score >= 650: return "Fair",      "🟠"
    if score >= 600: return "Poor",      "🔴"
    return "Very Poor", "🔴"

def individual_risk(age, monthly_income, required_loan, missed_payment,
                    existing_loan, years_employment, has_insurance):
    prob = 0.15
    if age < 25:            prob += 0.10
    elif age > 55:          prob += 0.05

    lti = required_loan / (monthly_income + 1)
    if lti > 10:            prob += 0.20
    elif lti > 5:           prob += 0.10
    elif lti < 2:           prob -= 0.05

    if missed_payment == "Yes, multiple times":  prob += 0.25
    elif missed_payment == "Yes, once or twice": prob += 0.12

    if existing_loan == "Yes, heavy burden":     prob += 0.15
    elif existing_loan == "Yes, manageable":     prob += 0.05

    if years_employment < 1:    prob += 0.15
    elif years_employment < 3:  prob += 0.05
    elif years_employment >= 5: prob -= 0.05

    if has_insurance:           prob -= 0.05
    return max(0.02, min(0.97, prob))

def msme_risk(owner_age, monthly_income, business_age, gst_registered,
              monthly_revenue, profit_margin_pct, required_loan,
              missed_payment, existing_loan, years_employment, has_insurance):
    prob = 0.20
    if business_age < 1:       prob += 0.20
    elif business_age < 3:     prob += 0.10
    elif business_age >= 5:    prob -= 0.08

    if not gst_registered:     prob += 0.12

    ltr = required_loan / (monthly_revenue + 1)
    if ltr > 12:   prob += 0.20
    elif ltr > 6:  prob += 0.10
    elif ltr < 2:  prob -= 0.05

    if profit_margin_pct < 5:     prob += 0.15
    elif profit_margin_pct < 15:  prob += 0.05
    elif profit_margin_pct >= 25: prob -= 0.08

    if missed_payment == "Yes, multiple times":  prob += 0.20
    elif missed_payment == "Yes, once or twice": prob += 0.10

    if existing_loan == "Yes, heavy burden":     prob += 0.12
    elif existing_loan == "Yes, manageable":     prob += 0.04

    if years_employment < 1:    prob += 0.08
    elif years_employment >= 5: prob -= 0.04

    if has_insurance:           prob -= 0.05
    return max(0.02, min(0.97, prob))

# ─────────────────────────────────────────
# Visualization
# ─────────────────────────────────────────
def draw_gauge(credit_score):
    fig, ax = plt.subplots(figsize=(6, 2.2))
    bands = [
        (300, 500, "#d32f2f", "Very Poor"),
        (500, 580, "#e57373", "Poor"),
        (580, 650, "#ff9800", "Fair"),
        (650, 700, "#fff176", "Good"),
        (700, 750, "#66bb6a", "Very Good"),
        (750, 850, "#2e7d32", "Excellent"),
    ]
    for lo, hi, col, lbl in bands:
        ax.barh(0, hi - lo, left=lo, color=col, height=0.4)
        ax.text((lo + hi) / 2, -0.32, lbl, ha="center", fontsize=7)
    ax.axvline(credit_score, color="black", linewidth=2.5, linestyle="--")
    ax.text(credit_score, 0.28, str(credit_score),
            ha="center", fontweight="bold", fontsize=12)
    ax.set_xlim(300, 850)
    ax.set_yticks([])
    ax.set_xlabel("Credit Score")
    ax.set_title("Credit Score Range")
    plt.tight_layout()
    plt.show()

def build_input_df(age, monthly_income, required_loan, missed_payment, existing_loan):
    late_flag = 1 if missed_payment != "No" else 0
    severe    = 1 if missed_payment == "Yes, multiple times" else 0
    return pd.DataFrame([[
        min(monthly_income / (monthly_income + 1) * 0.3, 1.0),  # RevolvingUtilization
        age,                                                       # age
        required_loan / (monthly_income * 12 + 1),                # DebtRatio
        monthly_income,                                            # MonthlyIncome
        1 if existing_loan != "No" else 0,                        # OpenCreditLines
        severe,                                                    # 90DaysLate
        0,                                                         # RealEstateLoans
        late_flag,                                                 # 30-59DaysLate
        0,                                                         # Dependents
        0,                                                         # 60-89DaysLate
        late_flag,                                                 # TotalLate
        late_flag + severe,                                        # DelinquencyScore
        required_loan / (monthly_income * 12 + 1) / (monthly_income + 1),  # DebtToIncome
        monthly_income,                                            # IncomePerDependent
    ]], columns=[
        "RevolvingUtilizationOfUnsecuredLines", "age", "DebtRatio",
        "MonthlyIncome", "NumberOfOpenCreditLinesAndLoans",
        "NumberOfTimes90DaysLate", "NumberRealEstateLoansOrLines",
        "NumberOfTime30-59DaysPastDueNotWorse", "NumberOfDependents",
        "NumberOfTime60-89DaysPastDueNotWorse",
        "TotalLate", "DelinquencyScore", "DebtToIncome", "IncomePerDependent",
    ])

def show_ml_charts(input_df):
    if not model_loaded or model is None:
        print("ℹ️ Model not available — ML charts unavailable.")
        return

    try:
        from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
        import shap

        # Note: We rely on the model being loaded. We re-load test data briefly for charts if needed
        # For simplicity in Notebook, we will just show SHAP for the specific input
        print("🔍 Computing SHAP values...")
        
        # Build explanation
        input_aligned = input_df.reindex(columns=model.feature_names_in_, fill_value=0)
        input_array = input_aligned.values.astype(np.float64)

        explainer = shap.TreeExplainer(model)
        shap_vals = np.array(explainer.shap_values(input_array), dtype=np.float64)
        
        # Handle expected_value formats
        raw_base = explainer.expected_value
        if isinstance(raw_base, (list, np.ndarray)):
            base_val = float(np.array(raw_base).flat[0])
        else:
            try:
                base_val = float(str(raw_base).strip("[]"))
            except:
                base_val = 0.5 # Fallback

        explanation = shap.Explanation(
            values        = shap_vals[0],
            base_values   = base_val,
            data          = input_array[0],
            feature_names = input_aligned.columns.tolist(),
        )
        
        plt.figure()
        shap.waterfall_plot(explanation, show=True)

    except Exception as e:
        print(f"SHAP Error: {e}")

# ─────────────────────────────────────────
# PDF Generator
# ─────────────────────────────────────────
def generate_pdf(data: dict, prob, prediction, credit_score, band, app_type):
    try:
        from reportlab.lib.pagesizes import A4
        from reportlab.lib import colors
        from reportlab.lib.units import cm
        from reportlab.platypus import (SimpleDocTemplate, Paragraph, Spacer,
                                        Table, TableStyle, HRFlowable)
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.enums import TA_CENTER
    except ImportError:
        print("ReportLab not installed. Skipping PDF.")
        return None

    filename = f"credit_report_{data.get('PAN', 'UNKNOWN')}.pdf"
    doc = SimpleDocTemplate(filename, pagesize=A4,
                            topMargin=2*cm, bottomMargin=2*cm,
                            leftMargin=2*cm, rightMargin=2*cm)
    styles  = getSampleStyleSheet()
    title_s = ParagraphStyle("t", parent=styles["Title"], fontSize=18, spaceAfter=4, alignment=TA_CENTER)
    sub_s   = ParagraphStyle("s", parent=styles["Normal"], fontSize=10, textColor=colors.grey, alignment=TA_CENTER)
    head_s  = ParagraphStyle("h", parent=styles["Heading2"], fontSize=12, spaceBefore=12, spaceAfter=4)
    disc_s  = ParagraphStyle("d", parent=styles["Normal"], fontSize=8, textColor=colors.grey, alignment=TA_CENTER)

    now    = datetime.datetime.now().strftime("%d %B %Y, %I:%M %p")
    result = "LOW RISK ✓" if prediction == "Low Risk" else "HIGH RISK ✗"
    rcol   = colors.green if prediction == "Low Risk" else colors.red

    elems = []
    elems.append(Paragraph("AI Credit Risk Evaluation Report", title_s))
    elems.append(Paragraph(f"Type: {app_type}  |  Generated on {now}", sub_s))
    elems.append(Spacer(1, 0.3*cm))
    elems.append(HRFlowable(width="100%", thickness=1, color=colors.grey))
    elems.append(Spacer(1, 0.3*cm))

    elems.append(Paragraph("Credit Summary", head_s))
    s_tbl = Table(
        [["Credit Score", "Band", "Default Probability", "Decision"],
         [str(credit_score), band, f"{round(prob*100,1)} %", result]],
        colWidths=[4*cm, 4*cm, 5*cm, 5*cm]
    )
    s_tbl.setStyle(TableStyle([
        ("BACKGROUND", (0,0),(-1,0), colors.HexColor("#1E3A5F")),
        ("TEXTCOLOR",  (0,0),(-1,0), colors.white),
        ("FONTNAME",   (0,0),(-1,0), "Helvetica-Bold"),
        ("ALIGN",      (0,0),(-1,-1),"CENTER"),
        ("GRID",       (0,0),(-1,-1), 0.5, colors.lightgrey),
        ("PADDING",    (0,0),(-1,-1), 8),
        ("TEXTCOLOR",  (3,1),(3,1),   rcol),
        ("FONTNAME",   (3,1),(3,1),   "Helvetica-Bold"),
    ]))
    elems.append(s_tbl)
    elems.append(Spacer(1, 0.3*cm))

    elems.append(Paragraph("Applicant Details", head_s))
    d_tbl = Table(
        [["Field", "Value"]] + [[k, str(v)] for k, v in data.items()],
        colWidths=[9*cm, 9*cm]
    )
    d_tbl.setStyle(TableStyle([
        ("BACKGROUND",    (0,0),(-1,0), colors.HexColor("#EEF2FF")),
        ("FONTNAME",      (0,0),(-1,0), "Helvetica-Bold"),
        ("GRID",          (0,0),(-1,-1), 0.5, colors.lightgrey),
        ("PADDING",       (0,0),(-1,-1), 6),
    ]))
    elems.append(d_tbl)
    elems.append(Spacer(1, 0.3*cm))
    elems.append(HRFlowable(width="100%", thickness=0.5, color=colors.grey))
    elems.append(Spacer(1, 0.2*cm))
    elems.append(Paragraph("Disclaimer: This report is AI-generated.", disc_s))

    doc.build(elems)
    return filename

In [5]:
# ─────────────────────────────────────────
# Interface Construction
# ─────────────────────────────────────────

# Output area for results
out = widgets.Output()

# --- Shared Widgets ---
w_type = widgets.ToggleButtons(options=['Individual', 'MSME'], description='Applicant:', style={'button_width':'100px'})
w_pan = widgets.Text(description='PAN:', placeholder='ABCDE1234F')
w_missed = widgets.Dropdown(options=["No", "Yes, once or twice", "Yes, multiple times"], description='Missed Pay:')
w_existing = widgets.Dropdown(options=["No", "Yes, manageable", "Yes, heavy burden"], description='Existing Loan:')
w_insurance = widgets.Checkbox(value=False, description='Has Insurance')
btn_calc = widgets.Button(description="Evaluate Risk", button_style='primary', icon='calculator')

# --- Individual Widgets ---
w_name_ind = widgets.Text(description='Name:')
w_age_ind = widgets.IntText(value=30, description='Age:')
w_income_ind = widgets.FloatText(value=40000, description='Income (₹):')
w_loan_ind = widgets.FloatText(value=200000, description='Loan (₹):')
w_emp_ind = widgets.IntText(value=3, description='Yrs Emp:')

box_individual = widgets.VBox([
    widgets.HTML("<h3>👤 Individual Details</h3>"),
    w_name_ind, w_pan, w_age_ind, w_income_ind, w_loan_ind,
    w_missed, w_existing, w_emp_ind, w_insurance
])

# --- MSME Widgets ---
w_name_msme = widgets.Text(description='Owner Name:')
w_age_msme = widgets.IntText(value=35, description='Owner Age:')
w_income_msme = widgets.FloatText(value=50000, description='Owner Income:')
w_biz_age = widgets.IntText(value=3, description='Biz Age:')
w_gst = widgets.Dropdown(options=['Yes', 'No'], description='GST Reg:')
w_rev = widgets.FloatText(value=200000, description='Revenue (₹):')
w_profit = widgets.IntSlider(value=15, min=0, max=100, description='Margin %:')
w_loan_msme = widgets.FloatText(value=500000, description='Loan (₹):')
w_emp_msme = widgets.IntText(value=3, description='Yrs Running:')

box_msme = widgets.VBox([
    widgets.HTML("<h3>🏢 MSME Details</h3>"),
    w_name_msme, w_pan, w_age_msme, w_income_msme,
    w_biz_age, w_gst, w_rev, w_profit, w_loan_msme,
    w_missed, w_existing, w_emp_msme, w_insurance
])

# Layout Container
form_container = widgets.VBox([box_individual])

def on_type_change(change):
    if change['new'] == 'Individual':
        form_container.children = [box_individual]
    else:
        form_container.children = [box_msme]

w_type.observe(on_type_change, names='value')

# ─────────────────────────────────────────
# Main Execution Logic
# ─────────────────────────────────────────
def run_evaluation(b):
    out.clear_output()
    with out:
        app_type = w_type.value
        pan_val = w_pan.value.strip().upper()

        if not validate_pan(pan_val):
            print("❌ Invalid PAN Format. Must be ABCDE1234F.")
            return

        if app_type == "Individual":
            prob = individual_risk(
                w_age_ind.value, w_income_ind.value, w_loan_ind.value,
                w_missed.value, w_existing.value, w_emp_ind.value, w_insurance.value
            )
            input_df = build_input_df(
                w_age_ind.value, w_income_ind.value, w_loan_ind.value,
                w_missed.value, w_existing.value
            )
            data_dict = {
                "Name": w_name_ind.value, "PAN": pan_val, "Age": w_age_ind.value,
                "Income": w_income_ind.value, "Loan": w_loan_ind.value
            }
        else:
            prob = msme_risk(
                w_age_msme.value, w_income_msme.value, w_biz_age.value,
                (w_gst.value == 'Yes'), w_rev.value, w_profit.value, w_loan_msme.value,
                w_missed.value, w_existing.value, w_emp_msme.value, w_insurance.value
            )
            input_df = build_input_df(
                w_age_msme.value, w_income_msme.value, w_loan_msme.value,
                w_missed.value, w_existing.value
            )
            data_dict = {
                "Owner": w_name_msme.value, "PAN": pan_val, "Biz Age": w_biz_age.value,
                "Revenue": w_rev.value, "Loan": w_loan_msme.value
            }

        credit_score = compute_credit_score(prob)
        band, emoji  = score_band(credit_score)
        prediction   = "Low Risk" if prob < 0.5 else "High Risk"
        
        # Display Text Results
        print(f"📊 Credit Score: {credit_score} | {emoji} {band}")
        print(f"📉 Default Probability: {round(prob*100, 1)}%")
        print(f"📝 Decision: {prediction}")
        
        # Draw Gauge
        draw_gauge(credit_score)
        
        # Draw ML Charts (SHAP)
        print("-" * 30)
        show_ml_charts(input_df)
        
        # Generate PDF
        print("-" * 30)
        pdf_file = generate_pdf(data_dict, prob, prediction, credit_score, band, app_type)
        if pdf_file:
            print(f"📄 PDF Report Generated: {pdf_file}")
            # Display link if running locally/binder
            display(FileLink(pdf_file))

btn_calc.on_click(run_evaluation)

# Display Full App
display(widgets.VBox([
    widgets.HTML("<h1>🏦 AI Credit Risk System</h1>"),
    w_type,
    widgets.HTML("<hr>"),
    form_container,
    widgets.HTML("<br>"),
    btn_calc,
    widgets.HTML("<hr>"),
    out
]))